In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from random import choice

df = pd.read_csv('bank_data.csv')
df['ratio'] = df['money'] / df['time (hr)']

data = df.to_numpy()
data_by_ratio = data[data[:,2].argsort()]
coords = df.filter(['x_coordinate', 'y_coordinate']).to_numpy()
closests = np.argsort(distance.squareform(distance.pdist(coords)), axis=1)

In [16]:
import numpy as np
from random import choice
from copy import deepcopy
from scipy.spatial import distance

id_index = 0
x_cord_index = 1
y_cord_index = 2
money_index = 3
time_index = 4
ratio_index = 5
speed = 30
simulations_amout = 100
simulation_depth = 100

actions = ['random', money_index, ratio_index]

class Game:
    def __init__(self, restore=False, state={}):
        self.data = data

        if(restore):
            local_state = deepcopy(state)
            self.visited = local_state['visited']
            self.score = local_state['score']
            self.time_left = local_state['time_left']
            self.time_to_exit = local_state['time_to_exit']
        else:
            self.visited = [choice(self.data)] 
            self.score = self.visited[0][money_index]
            self.actions = ['random']
            self.time_left = 24 - self.visited[0][time_index]
            self.time_to_exit = distance.euclidean(self.visited[-1][x_cord_index:y_cord_index], [0,0]) / speed
        self.visited_dict = {}

        for visit in self.visited:
            self.visited_dict[visit[id_index]] = True

        self.game_ended = False

    def get_state(self):
        return {
            'visited': self.visited,
            'score': self.score,
            'time_left': self.time_left,
            'time_to_exit': self.time_to_exit
        }

    def get_visited_ids(self):
        return list(map(lambda x: x[id_index], self.visited))

    def play_move(self, move):
        # don't let move when game is over
        if (self.time_to_exit > self.time_left):
            self.game_ended = True
            return

        next_move = move
        distance_to_next = distance.euclidean(self.visited[-1][x_cord_index:y_cord_index], 
                            next_move[x_cord_index:y_cord_index])
        self.visited.append(next_move)
        self.visited_dict[next_move[id_index]] = True
        # update the score
        self.score = self.score + next_move[money_index]
        self.time_left = self.time_left - next_move[time_index] - (distance_to_next / speed)
        self.time_to_exit = distance.euclidean(self.visited[-1][x_cord_index:y_cord_index], [0,0]) / speed

In [14]:
class Simulated_Game(Game):
    def play_random_game(self):
            while(self.time_to_exit < self.time_left):
                self.play_move(self.choose_move_type())
            self.visited = self.visited[:-1]
            # game over 
            self.game_ended = True
    
    def choose_move_type(self):
        move_type = choice(actions)
        self.actions.append(move_type)
        distances_from_next = closests[int(self.visited[-1][id_index])]
        closest_values = [self.data[i] for i in distances_from_next[:simulation_depth] 
                            if self.data[i][id_index] not in self.visited_dict]

        if (move_type == 'random'):
            return choice(closest_values)
        # return the highest value based on the current move type selected
        return closest_values[np.argmax(closest_values, axis=0)[move_type]]

# helpers
def get_current_policy(ratio_score, random_score):
    maximum = max(ratio_score, random_score)

    if(maximum == ratio_score):
        return ratio_index

    if(maximum == random_score):
        return 'random'

In [13]:
def rob_simulator():
    rob = Simulated_Game()
    for i in range(10000):
            temp_game = Simulated_Game()
            temp_game.play_random_game()
            if(temp_game.score > rob.score):
                rob = temp_game
    return rob


g = rob_simulator()
print(g.get_visited_ids(), "${:,.2f}".format(g.score))

[428.0, 2707.0, 7102.0, 1676.0, 3193.0, 370.0, 4042.0, 4006.0, 7343.0, 487.0, 8525.0, 3926.0, 6591.0, 1424.0, 1356.0, 4628.0, 6216.0, 3186.0, 4234.0, 1304.0, 518.0, 8690.0, 4937.0, 6212.0, 8588.0, 9089.0, 8550.0, 5142.0, 8912.0, 5548.0, 6528.0, 5507.0, 781.0, 9653.0, 8381.0, 9602.0, 6759.0, 1997.0, 5150.0, 8796.0, 6104.0, 1702.0, 4293.0, 4271.0, 3025.0, 268.0, 433.0, 4610.0, 6635.0, 7255.0, 9546.0, 2.0, 3613.0, 8696.0, 58.0, 1372.0, 3716.0, 406.0] $4,025,100.00
